In [ ]:
#hide
from nbdev import *
%nbdev_default_export release

Cells will be exported to nbdev.release,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Create release notes

> Release notes auto-generated from GitHub issues

In [ ]:
#export
from nbdev.imports import *
from fastcore.all import *
from nbdev.export import *
from datetime import datetime
from textwrap import fill
import requests

In [ ]:
#export
GH_HOST = "https://api.github.com"

In [ ]:
#export
def _issue_txt(issue):
    res = f'- {issue["title"].strip()} ([#{issue["number"]}]({issue["url"]}))\n'
    body = issue['body']
    if not body: return res
    return res + fill(body.strip(), initial_indent="  - ", subsequent_indent="    ") + "\n"

def _issues_txt(iss, label):
    if not iss: return ''
    res = f"### {label}\n\n"
    return res + '\n'.join(map(_issue_txt, iss))

In [ ]:
#export
class FastRelease:
    def __init__(self, owner=None, repo=None, token=None, **groups):
        "Create CHANGELOG.md from GitHub issues"
        if not groups: groups = dict(breaking="Breaking Changes", enhancement="New Features", bug="Bugs Squashed")
        self.groups = groups
        cfg = Config()
        os.chdir(cfg.config_file.parent)
        if not owner: owner = cfg.user
        if not repo:  repo  = cfg.lib_name
        if not token: token = Path('token').read_text().strip()
        store_attr()
        self.session = requests.Session()
        self.session.headers['Authorization'] = f'token {token}'
        self.repo_url = f"{GH_HOST}/repos/{owner}/{repo}"

    def _tag_date(self, tag):
        tag_url = f"{self.repo_url}/git/ref/tags/{tag}"
        try: tag_d = self.session.get(tag_url).json()
        except HTTPError: raise Exception(f"Failed to find tag {tag}")
        commit_d = self.session.get(tag_d["object"]["url"]).json()
        self.commit_date = commit_d["committer"]["date"].replace('Z', '+00:00')
        return self.commit_date

    def _issues(self, label):
        return self.session.get(f"{self.repo_url}/issues?state=closed&sort=created&filter=all&"+
                            f"since={self.commit_date}&labels={label}").json()
    
    def _issue_groups(self):
        return parallel(self._issues, self.groups.keys(), progress=False).map(L)    
    
    def _latest_release(self):
        rel_url = f"{self.repo_url}/releases/latest"
        return self.session.get(rel_url).json()["tag_name"]

    def changelog(self, debug=False):
        "Create the CHANGELOG.md file, or return the proposed text if `debug` is `True`"
        fn = 'CHANGELOG.md'
        shutil.copy(fn, fn+".bak")
        txt = Path(fn).read_text()
        marker = '<!-- do not remove -->\n'
        ver = self._latest_release()
        self._tag_date(ver)
        res = f"## {ver}\n"
        issues = self._issue_groups()
        res += '\n'.join(L(issues, self.groups.values()).zip().starmap(_issues_txt))
        res = txt.replace(marker, marker+res)
        if debug: return res
        else: Path(fn).write_text(res)

To create a markdown changelog, first create a `FastRelease` object, passing a mapping from GitHub labels to markdown titles. Put your github token in a file named `token` at the root of your repo.

In [ ]:
# rel = FastRelease(breaking="Breaking Changes", enhancement="New Features", bug="Bugs Squashed")

In [ ]:
show_doc(FastRelease.changelog)

<h4 id="FastRelease.changelog" class="doc_header"><code>FastRelease.changelog</code><a href="__main__.py#L36" class="source_link" style="float:right">[source]</a></h4>

> <code>FastRelease.changelog</code>(**`debug`**=*`False`*)

Create the CHANGELOG.md file, or return the proposed text if `debug` is `True`

In [ ]:
# print(rel.changelog(debug=True))

## Export-

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 05a_conda.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 10_release.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted magic_flags.ipynb.
Converted nbdev_callbacks.ipynb.
Converted tutorial.ipynb.
